In [1]:
import numpy as np
import sys
import os

from mts import (
    LabsEnergyCounter,
    merit_factor,
    combine,
    mutate,
    tabu_search,
    memetic_tabu_search
)

labs_energy = LabsEnergyCounter(warn_at=200_000)


In [ ]:
# Parallelize across N using process-based parallelism (fork) for reproducibility & isolation

Ns = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 66, 70, 75, 80, 82]
results = []

# Problem parameters
pop_size = 20  # Population size
max_generations = 1000  # Number of generations

# seeds = 42, 7, 67, 58, 7841, 1234, 5678, 91011
seeds = [1234, 5678, 91011]


def _run_one_N(N: int, seed: int):
    np.random.seed(int(seed))

    labs_energy.reset()

    best_s, best_energy, population, energies = memetic_tabu_search(
        N=int(N),
        energy_func=labs_energy,
        pop_size=pop_size,
        max_generations=max_generations,
        p_mut=0.1,
        p_combine=0.5,
        tabu_max_iter=100,
        tabu_tenure=7,
        verbose=False
    )

    return {
        "seed": int(seed),
        "N": int(N),
        "best_energy": int(best_energy),
        "merit_factor": float(merit_factor(best_s, best_energy)),
        "energy_evals": int(labs_energy.count),
        "best_s": best_s.copy(),
        "final_population": population,
        "final_energies": energies
    }


import multiprocessing as mp
import concurrent.futures as cf

print("Running Memetic Tabu Search for LABS (parallel over N, loop over seeds)")
print(f"Population size: {pop_size}, Max generations: {max_generations}")
print(f"Seeds: {seeds}")
print("-" * 50)

_ctx = mp.get_context("fork")
max_workers = min(len(Ns), (os.cpu_count() or 1))

for seed in seeds:
    seed_results = []

    with cf.ProcessPoolExecutor(max_workers=max_workers, mp_context=_ctx) as ex:
        futures = {ex.submit(_run_one_N, int(N), int(seed)): int(N) for N in Ns}
        for fut in cf.as_completed(futures):
            seed_results.append(fut.result())

    seed_results.sort(key=lambda r: Ns.index(r["N"]))
    results.extend(seed_results)

    print(f"\nSeed={seed} Summary (best_energy, merit_factor, evals):")
    for r in seed_results:
        print(f"N={r['N']}: E={r['best_energy']}, F={r['merit_factor']:.6f}, evals={r['energy_evals']}")


In [208]:
# Task: Export the current `results` list to a CSV file containing N, best_energy, merit_factor, energy_evals, and best_s.

import csv

out_path = "labs_mts_results2.csv"

with open(out_path, "w", newline="") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["N", "best_energy", "merit_factor", "energy_evals", "best_s"],
    )
    writer.writeheader()
    for r in results:
        best_s_arr = np.asarray(r.get("best_s", []))
        writer.writerow(
            {
                "N": int(r["N"]),
                "best_energy": int(r["best_energy"]),
                "merit_factor": float(r["merit_factor"]),
                "energy_evals": int(r["energy_evals"]),
                "best_s": " ".join(map(str, best_s_arr.tolist())),
            }
        )

print(f"Wrote {len(results)} rows to {out_path}")


Wrote 36 rows to labs_mts_results2.csv
